<a href="https://colab.research.google.com/github/mishra39/deep_rl_course_hf/blob/main/ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create a virtual display 🔽

During the notebook, we'll need to generate a replay video. To do so, with colab, **we need to have a virtual screen to be able to render the environment** (and thus record the frames).

Hence the following cell will install the librairies and create and run a virtual screen 🖥

In [4]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay
!apt install x11-utils
!pip install pyglet

In [5]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [6]:
# Display video
import glob
import io
import base64
from IPython.display import HTML

def show_video():
    """Embeds the recorded video in the notebook output."""
    mp4list = glob.glob('videos/*.mp4')
    if len(mp4list) > 0:
        video = mp4list[0]
        with io.open(video, 'r+b') as f:
            encoded = base64.b64encode(f.read()).decode()
        # Create an HTML display object for Colab
        return HTML(data=f'<video width="1000" controls><source src="data:video/mp4;base64,{encoded}" type="video/mp4" /></video>')
    else:
        print("No video files found in the 'videos' directory.")

### Install dependencies 🔽

The first step is to install the dependencies, we’ll install multiple ones:
- `gymnasium`
- `panda-gym`: Contains the robotics arm environments.
- `stable-baselines3`: The SB3 deep reinforcement learning library.
- `huggingface_sb3`: Additional code for Stable-baselines3 to load and upload models from the Hugging Face 🤗 Hub.
- `huggingface_hub`: Library allowing anyone to work with the Hub repositories.

⏲ The installation can **take 10 minutes**.

In [1]:
#!pip install stable-baselines3[extra]
!pip install gymnasium

In [4]:
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.7-cp312-cp312-linux_x86_64.whl size=99873169 sha256=224e1a81008f9275284fd671c5c574287385aff35ae4e1fccf875693d912fb68
  Stored in directory: /root/.cache/pip/wheels/72/95/1d/b336e5ee612ae9a019bfff4dc0bedd100ee6f0570db205fdf8
Successfully built pybullet


## W&B Prerequisites

Install the W&B Python SDK and log in:

In [2]:
!pip install wandb -qU
!pip install -q gym numpy tensorboard
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [3]:
# Log in to your W&B account
import wandb
import random
import math

In [7]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mishra39 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Setup

In [7]:
import argparse
import os
import random
import time
from distutils.util import strtobool

import gymnasium as gym  # Use gymnasium instead of gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

## Arguments

In [8]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class PPOConfig:
    # Experiment settings
    exp_name: str = "ppo_experiment"
    gym_id: str = "CartPole-v1"
    learning_rate: float = 2.5e-4
    seed: int = 1
    total_timesteps: int = 25000
    torch_deterministic: bool = True
    cuda: bool = True
    track: bool = False
    wandb_project_name: str = "ppo-implementation-details"
    wandb_entity: Optional[str] = None
    capture_video: bool = False

    # Algorithm specific arguments
    num_envs: int = 4
    num_steps: int = 128
    anneal_lr: bool = True
    gae: bool = True
    gamma: float = 0.99
    gae_lambda: float = 0.95
    num_minibatches: int = 4
    update_epochs: int = 4
    norm_adv: bool = True
    clip_coef: float = 0.2
    clip_vloss: bool = True
    ent_coef: float = 0.01
    vf_coef: float = 0.5
    max_grad_norm: float = 0.5
    target_kl: Optional[float] = None

    def __post_init__(self):
        # Computed values
        self.batch_size = int(self.num_envs * self.num_steps)
        self.minibatch_size = int(self.batch_size // self.num_minibatches)

# Create instance with default values
# args = PPOConfig()

# Or customize specific values
# args = PPOConfig(learning_rate=1e-3, num_envs=8, total_timesteps=50000)

# print(f"Batch size: {args.batch_size}")
# print(f"Minibatch size: {args.minibatch_size}")

## Gym Envrionment

In [9]:
def make_env(gym_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk

## Main Loop

In [10]:
args = PPOConfig()
run_name = f"{args.gym_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
if args.track:
    wandb.init(
        project=args.wandb_project_name,
        entity=args.wandb_entity,
        sync_tensorboard=True,
        config=vars(args),
        name=run_name,
        monitor_gym=True,
        save_code=True,
    )
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
)

# TRY NOT TO MODIFY: seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

In [15]:
print("Using device:", device)
# env setup
env = gym.make(args.gym_id, render_mode="rgb_array")
env = gym.wrappers.RecordEpisodeStatistics(env)
env = gym.wrappers.RecordVideo(
        env,
        video_folder="videos",
        episode_trigger=lambda episode_id: episode_id % 100 == 0,  # Record every 100 episodes
        name_prefix="ppo"
    )
observation = env.reset()

for _ in range(300):
  action = env.action_space.sample()
  observation, reward, terminated, truncated, info = env.step(action)
  done = terminated or truncated
  if done:
    observation = env.reset()
    print(f"episodic_return: {info['episode']['r']}")
    episodic_return = 0
env.close()
# Display the recorded video in Colab
show_video()

Using device: cpu
episodic_return: 17.0
episodic_return: 30.0
episodic_return: 18.0
episodic_return: 13.0
episodic_return: 22.0
episodic_return: 22.0
episodic_return: 11.0
episodic_return: 10.0
episodic_return: 20.0
episodic_return: 14.0
episodic_return: 24.0
episodic_return: 29.0
episodic_return: 21.0
episodic_return: 12.0
episodic_return: 28.0
